In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
import pickle
import pandas as pd
from datetime import datetime
import csv

# Project Lists

In [ ]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.ma   ximize_window()
# Extract link and project name
i = 1
while True:
    try:
        website = "https://cryptorank.io/funding-rounds?page={0}".format(i)
        
        driver.get(website)
        time.sleep(2)
        projects = driver.find_elements(By.XPATH,'//tr[@class="sc-7ff8d1ea-0 kuuWTw init-scroll"]')
        
        for project in projects:    
            name = project.find_element(By.TAG_NAME, "p").text
            token_sale_link = project.find_element(By.TAG_NAME, "a").get_attribute("href")
            overview_link = token_sale_link.replace("/ico/","/price/")
            
            data.append([name, overview_link, token_sale_link])
        i += 1
    except:
        break

df = pd.DataFrame(data, columns=['First Funding date', 'Overview', 'Token Sale'])
df['Name'] = df['Overview'].apply(lambda x: x.split("/")[-1]).apply(lambda x: x.split("#")[0]).apply(lambda x: x.replace("-", " ").title())

def convert_date(date_str):
    if len(date_str.split()) == 2:
        date_str += " 2024"
    try:
        dt = datetime.strptime(date_str, '%d %b %Y')
        return dt.strftime('%Y-%m-%d')
    except ValueError:
        return None

df['First Funding date'] = df['First Funding date'].apply(convert_date)
df.drop_duplicates().to_csv("project_lists.csv", index = False)

# Project Information

## Overview

In [2]:
df = pd.read_csv("../static/Data/project_lists.csv")
df.head()

,First Funding date,Overview,Token Sale,Name
0,2024-04-03,https://cryptorank.io/price/raiinmaker,https://cryptorank.io/ico/raiinmaker,Raiinmaker
1,2024-04-03,https://cryptorank.io/price/satoshisync#fundin...,https://cryptorank.io/ico/satoshisync#funding-...,Satoshisync
2,2024-04-03,https://cryptorank.io/price/legitimate,https://cryptorank.io/ico/legitimate,Legitimate
3,2024-04-03,https://cryptorank.io/price/mangata,https://cryptorank.io/ico/mangata,Mangata
4,2024-04-02,https://cryptorank.io/price/bonus-block#fundin...,https://cryptorank.io/ico/bonus-block#funding-...,Bonus Block


In [ ]:

# Extraction of link and project name ends here

services = []
total_raised_s = []
descriptions = []
links_s = []
name = []



for website in df['Overview'][18:300]:
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    
    extracted_data = []
    crypto_name = website.split("/")[-1].split("#")[0].replace("_"," ").title()
    name.append(crypto_name)
    extracted_data.append(crypto_name)
    
    driver.get(website)
    time.sleep(5)
    service, total_raised, description, links = '', '', '', []
    try:
        service_element = driver.find_element(By.XPATH,'//a[starts-with(@href, "/categories/")]/p')
        services.append(service_element.text)
        extracted_data.append(service_element.text)
    except:
        services.append("")
        extracted_data.append("")
    try:
        total_raised_element = driver.find_element(By.XPATH,'//p[@class="sc-50f3633f-0 gsUEex"]')
        total_raised_s.append(total_raised_element.text)
        extracted_data.append(total_raised_element.text)
    except:
        total_raised_s.append("")
        extracted_data.append("")
    try:
        description_element = driver.find_element(By.XPATH,'//div[@class="sc-50f3633f-0 sc-f084fb54-0 eEpKHS gQIZzl"]/p')
        descriptions.append(description_element.text)
        extracted_data.append(description_element.text)
    except:
        descriptions.append("")
        extracted_data.append("")
    
    
    ###########################
    
    try:
        button = driver.find_elements(By.XPATH,'//button[@class="sc-4e8da034-0 cHRla-d"]')
        for i in range(len(button)):
            driver.get(website)
            time.sleep(5)
            button = driver.find_elements(By.XPATH,'//button[@class="sc-4e8da034-0 cHRla-d"]')
            link = button[i]
            time.sleep(2)
            link.click()
            time.sleep(5)
            link_elements = driver.find_elements(By.XPATH,'//div[@class="sc-f30b0320-0 cAUFgN"]')
            heading = []
            some_links_text = []
            some_links = []
            for link_element in link_elements:
                some_links.append([i.get_attribute("href") for i in link_element.find_elements(By.TAG_NAME,"a")])
            links.append([some_links])
        links_s.append(links)
        extracted_data.append(links)
    except:
        links_s.append("")
        extracted_data.append("")
        pass
    
    ############################
    
    with open("output.csv", "a") as fp:
        wr = csv.writer(fp, dialect='excel')
        wr.writerow(extracted_data)

    print(len(name),len(services),len(total_raised_s),len(descriptions),len(links_s))
    
    time.sleep(15)
    driver.quit()

In [ ]:
print("done")

In [ ]:
# data = [name,services,total_raised_s,descriptions,links_s]
# df_overview = pd.DataFrame(data).T
# df_overview.columns = ["Crypto Name", "Service", "Total Raised", "Description", "Links"]
# df_overview.head()
# df_overview.to_csv("Overview_details.csv", index=False)

## Fundraising

In [ ]:
df = pd.read_csv("../static/Data/project_lists.csv")
df.head()

In [5]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
data = []
for website in df['Token Sale'][185:300]:
    crypto_name = website.split("/")[-1].split("#")[0].replace("_"," ").title()
    driver.get(website)
    item_elements = driver.find_elements(By.XPATH,'//div[@class="sc-e007581-0 sc-b608d829-0 kDrqot bslrEt"]')
    data.append([crypto_name,[div.text for div in item_elements]])
    time.sleep(5)

driver.quit()

df_fundings = pd.DataFrame(data, columns = ["Name", "Funding Details"])

# df_fundings.to_csv("Funding_details.csv", index=False)
df_fundings.to_csv("Funding_details.csv", mode='a', index=False)

## Twitter

In [ ]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get(twitter_url) 
followers = driver.find_element(By.XPATH,'//a[contains(@href, "/verified_followers")]').text
followers

driver.quit()